# Head ripper tool to analyze topical relevance for On-Page SEO
Read the how to guide on http://bluerivermountains.com/en/head-ripper-method

In [ ]:
!pip install --upgrade gspread
!pip install requests
!pip install requests-HTML

import requests
from requests_html import HTMLSession

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
#@title Enter URLs to rip headers from
url1 = "" #@param {type:"string"}
url2 = "" #@param {type:"string"}
url3 = "" #@param {type:"string"}
url4 = "" #@param {type:"string"}
url5 = "" #@param {type:"string"}
url6 = "" #@param {type:"string"}
url7 = "" #@param {type:"string"}
url8 = "" #@param {type:"string"}
url9 = "" #@param {type:"string"}
url10 = "" #@param {type:"string"}

sheetName = "Head Ripper - Analysis of competitors subtopics" #@param {type:"string"}

rawList = [url1,url2,url3,url4,url5,url6,url7,url8,url9,url10]

#clean list from empty entries
urlList = list(filter(None, rawList))

## Get all headers

In [ ]:
headerBank = []
for url in urlList:
  try:
      session = HTMLSession()
      response = session.get(url)
      
  except requests.exceptions.RequestException as e:
      print(e)

  headerList =  response.html.find('h1, h2, h3, h4, h5, h6')
  titleTag = response.html.find('title')

  # create element in list for each url
  headerBank.append({
      "url": url,
      "title": titleTag[0].text,
      "headers": []
  })

  # add header data to each element
  for id, header in enumerate(headerList):
    headerBank[-1]["headers"].append({
          "headerIndex": id,
          "headerText": header.text,
          "headerType": header.tag
        })

# add data to Gsheet    
sh = gc.create(sheetName)

worksheet = gc.open(sheetName).sheet1

worksheet.update('A1', 'URL')
worksheet.update('A2', 'Title')


# load alphabet to iterate through
from string import ascii_lowercase

# insert urls
for j,page in enumerate(headerBank):
  worksheet.update(ascii_lowercase[j+1]+str(1), page['url'])

# insert titles
for j,page in enumerate(headerBank):
  worksheet.update(ascii_lowercase[j+1]+str(2), page['title'])

# insert headers  
for i,page in enumerate(headerBank):
  for j,header in enumerate(page['headers']):
    worksheet.update(ascii_lowercase[i+1]+str(j+3), header["headerType"] + " - " + header["headerText"])


    